In [ ]:
import requests
import pandas as pd

fmarket_headers = {
    "accept": "application/json, text/plain, */*",
    "accept-language": "vi",
    "content-type": "application/json",
    "sec-ch-ua": "\"Microsoft Edge\";v=\"119\", \"Chromium\";v=\"119\", \"Not?A_Brand\";v=\"24\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-site",
    "referrer": "https://fmarket.vn/"
}

payload = {
    "types": ["NEW_FUND", "TRADING_FUND"],
    "issuerIds": [],
    "sortOrder": "DESC",
    "sortField": "navTo6Months",
    "page": 1,
    "pageSize": 100,
    "isIpo": False,
    "fundAssetTypes": [],
    "bondRemainPeriods": [],
    "searchField": "",
    "isBuyByReward": False,
    "thirdAppIds": []
}

url = "https://api.fmarket.vn/res/products/filter"
response = requests.post(url, json=payload, headers=fmarket_headers)
status = response.status_code

if status != 200:
    raise requests.exceptions.HTTPError(
        f"Error in API response: {status} - {response.text}"
    )

data = response.json()
print('Total number of funds: ', data['data']['total'])

df = pd.json_normalize(data, record_path=['data', 'rows'])

# select columns to display
column_subset = [
    'id',
    'shortName',
    'name',
    'owner.name',
    'dataFundAssetType.name',
    'firstIssueAt',
    'managementFee',
    'nav',
    'productNavChange.navToPrevious',
    'productNavChange.navToLastYear',
    'productNavChange.navToBeginning',
    'productNavChange.navTo1Months',
    'productNavChange.navTo3Months',
    'productNavChange.navTo6Months',
    'productNavChange.navTo12Months',
    'productNavChange.navTo24Months',
    'productNavChange.navTo36Months',
    'productNavChange.annualizedReturn36Months',
    'productNavChange.updateAt'            
]
df = df[column_subset]

# sort by column
df = df.sort_values('shortName')

# remove index column
df = df.reset_index(drop=True)

# output
print(df.head(5))
#df.to_csv('fmarket.csv', index=False, encoding='utf-8')

# compare pyarrow vs numpy backend dataframe
df.info()
df = df.convert_dtypes(dtype_backend="pyarrow")
df.info()



In [1]:
from datetime import datetime
import requests

dnse_headers = {
  'authority': 'services.entrade.com.vn',
  'accept': 'application/json, text/plain, */*',
  'accept-language': 'en-US,en;q=0.9',
  'dnt': '1',
  'origin': 'https://banggia.dnse.com.vn',
  'referer': 'https://banggia.dnse.com.vn/',
  'sec-ch-ua': '"Edge";v="114", "Chromium";v="114", "Not=A?Brand";v="24"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"Windows"',
  'sec-fetch-dest': 'empty',
  'sec-fetch-mode': 'cors',
  'sec-fetch-site': 'cross-site',
  'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 Edg/114.0.1788.0'
}

start_date = "2023-01-01"
end_date = "2023-01-10"
symbol = "FPT"
interval = '1D'

start_timestamp = int(datetime.strptime(start_date, "%Y-%m-%d").timestamp())
end_timestamp = int(datetime.strptime(end_date, "%Y-%m-%d").timestamp())

# API call
url = f"https://services.entrade.com.vn/chart-api/v2/ohlcs/stock?from={start_timestamp}&to={end_timestamp}&symbol={symbol}&resolution={interval}"
#url = "https://services.entrade.com.vn/chart-api/v2/ohlcs/stock?from=1704076256&to=1704853856&symbol=VIB&resolution=1D"
response = requests.get(url, headers=dnse_headers)
if response.status_code != 200:
    raise requests.exceptions.HTTPError(
        f"Error in API response: {response.status_code} - {response.text}"
    )

data = response.json()

print(data)

{'t': [1672711200, 1672797600, 1672884000, 1672970400, 1673229600], 'o': [65.34, 68.65, 68.14, 68.31, 68.4], 'h': [67.97, 68.83, 69.24, 68.9, 68.74], 'l': [65.25, 67.37, 68.14, 68.14, 67.89], 'c': [67.97, 67.97, 68.48, 68.22, 68.22], 'v': [1535200, 794300, 1012600, 719400, 518900], 'nextTime': 0}
